In [ ]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms


class CustomImageDataset(Dataset):
    def read_data_set(self):

        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB")

        if self.transforms is not None:
            image = self.transforms(image)

        filename = os.path.basename(self.image_files_path[index])
        return {'image': image, 'label': self.labels[index], 'filename': filename}

    def __len__(self):
        return self.length


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

import torch
import torch.nn as nn

class ResidualSEBlock(nn.Module):
    """Residual block with Squeeze-and-Excitation attention"""
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels//16, 1),
            nn.ReLU(),
            nn.Conv2d(out_channels//16, out_channels, 1),
            nn.Sigmoid()
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        out = self.conv1(x)
        out = self.conv2(out)
        se_weight = self.se(out)
        out = out * se_weight
        out += residual
        return nn.LeakyReLU(0.1)(out)

class NormalConvBlock(nn.Module):
    """Standard Convolutional Block without Residual Connections"""
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.1),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.1)
        )

    def forward(self, x):
        return self.conv(x)

class CustomConvNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Alternating residual and normal blocks
        self.layer1 = self._make_layer(3, 16, 2, use_residual=True)    # Residual
        self.layer2 = self._make_layer(16, 32, 1, use_residual=False)  # Normal
        self.layer3 = self._make_layer(32, 64, 1, use_residual=True)   # Residual
        self.layer4 = self._make_layer(64, 128, 1, use_residual=False) # Normal
        self.layer5 = self._make_layer(128, 256, 1, use_residual=True) # Residual
        self.layer6 = self._make_layer(256, 512, 1, use_residual=False)# Normal

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.6),
            nn.Linear(256, num_classes)
        )

    def extract_features(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.global_avg_pool(x)
        return x.view(x.size(0), -1)  # Shape: [batch_size, 512]


    def _make_layer(self, in_channels, out_channels, stride, use_residual):
        """Factory method to create different block types"""
        if use_residual:
            block = ResidualSEBlock(in_channels, out_channels, stride)
        else:
            block = NormalConvBlock(in_channels, out_channels, stride)

        return nn.Sequential(
            block,
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.4 if out_channels > 64 else 0.3)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.global_avg_pool(out)
        out = out.view(out.size(0), -1)
        return self.classifier(out)


In [ ]:
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt

# Hyperparameters
hyper_param_epoch = 200
hyper_param_batch = 64
hyper_param_learning_rate = 0.001
weight_decay = 1e-4
patience = 100


# Data augmentation

# Enhanced Data Augmentation
transforms_train = transforms.Compose([
    transforms.Resize((245, 457)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(25),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.RandomAffine(0, shear=15, scale=(0.9, 1.1)),
    transforms.ToTensor()
])

transforms_test = transforms.Compose([
    transforms.Resize((245, 457)),
    transforms.ToTensor()
])


# Datasets and DataLoaders

train_data_set = CustomImageDataset(data_set_path="E://Project//PTE//trainingSet", transforms=transforms_train)
test_data_set = CustomImageDataset(data_set_path="E://Project//PTE//testSet", transforms=transforms_test)

# Define the validation split ratio
validation_ratio = 0.2

# Calculate the number of samples for training and validation
train_size = int((1 - validation_ratio) * len(train_data_set))
val_size = len(train_data_set) - train_size

# Split the dataset into training and validation
train_subset, val_subset = random_split(train_data_set, [train_size, val_size])

# Create DataLoader for validation set
val_loader = DataLoader(val_subset, batch_size=hyper_param_batch, shuffle=False)
train_loader = DataLoader(train_subset, batch_size=hyper_param_batch, shuffle=True)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=False)





In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_classes = train_data_set.num_classes
custom_model = CustomConvNet(num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)

# Modified training loop
best_accuracy = 0.0
no_improve = 0
train_losses = []
valid_losses = []

for epoch in range(hyper_param_epoch):
    custom_model.train()
    running_loss = 0.0

    # Training phase
    for i_batch, item in enumerate(train_loader):
        images = item['image'].to(device)
        labels = item['label'].to(device)

        outputs = custom_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(custom_model.parameters(), 5.0)
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)


    # Validation phase
    custom_model.eval()
    test_loss, correct, total = 0.0, 0, 0


    with torch.no_grad():
        for item in val_loader:
            images = item['image'].to(device)
            labels = item['label'].to(device)
            outputs = custom_model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss /= total
    valid_losses.append(test_loss)
    test_accuracy = 100 * correct / total

    # Update learning rate based on loss
    scheduler.step(test_loss)

    # Save model ONLY if accuracy improves
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        no_improve = 0
        torch.save(custom_model.state_dict(), 'best_model.pth')
        print(f"▲ New best model saved with accuracy: {test_accuracy:.2f}%")
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"▼ Early stopping at epoch {epoch+1} (No improvement for {patience} epochs)")
            break

    # Print epoch stats
    print(f'Epoch [{epoch+1}/{hyper_param_epoch}] | '
          f'Train Loss: {train_loss:.4f} | '
          f'Test Loss: {test_loss:.4f} | '
          f'Test Accuracy: {test_accuracy:.2f}% | '
          f'Best Accuracy: {best_accuracy:.2f}% | '
          f'LR: {optimizer.param_groups[0]["lr"]:.1e}')

# Load best model for final evaluation
custom_model.load_state_dict(torch.load('best_model.pth'))
print(f"\nTraining complete. Best accuracy: {best_accuracy:.2f}%")


plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns

# --- Settings ---
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_classes = 3
class_names = ["normal", "NSCLC", "SCC"]

# --- Load Model ---
model = CustomConvNet(num_classes=num_classes)
model.load_state_dict(torch.load('best_best_model.pth', map_location=device))
model.to(device)
model.eval()

# --- Gather Predictions and Labels ---
all_labels = []
all_preds = []
all_probs = []
all_filename = []
with torch.no_grad():
    for batch in test_loader:
        images = batch['image'].to(device)
        labels = batch['label'].cpu().numpy()
        filename = batch['filename']
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        preds = np.argmax(probs, axis=1)
        all_labels.extend(labels)
        all_preds.extend(preds)
        all_probs.extend(probs)
        all_filename.extend(filename)

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
all_probs = np.array(all_probs)

# --- Confusion Matrix ---
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Pretty confusion matrix
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# --- Per-Class Accuracy ---
class_acc = cm.diagonal() / cm.sum(axis=1)
print("\nPer-Class Accuracy:")
for i, acc in enumerate(class_acc):
    print(f"{class_names[i]}: {acc:.2%}")

# --- Classification Report ---
report = classification_report(all_labels, all_preds, target_names=class_names)
print("\nClassification Report:")
print(report)

# --- ROC Curves and AUC ---
labels_binarized = label_binarize(all_labels, classes=np.arange(num_classes))

plt.figure(figsize=(8, 7))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(labels_binarized[:, i], all_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curves')
plt.legend(loc="lower right")
plt.show()


In [ ]:
import pandas as pd

df = pd.DataFrame({
    'all_filename': all_filename,
    'all_preds': all_preds,
    'all_labels':all_labels
})

# Save the DataFrame to a CSV file without the index
df.to_csv('predictions1.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# Load the CSV file
file_path = "predictions1.csv"  # Adjust this path if necessary
df = pd.read_csv(file_path, header=None, names=["Filename", "Predicted", "True"])

# Extract patient number from filename
df["PatientID"] = df["Filename"].str.extract(r"(\d{5})")
df = df.dropna(subset=["PatientID"])
df["PatientID"] = df["PatientID"].astype(int)

# Majority voting per patient
patient_predictions = df.groupby("PatientID")["Predicted"].agg(lambda x: x.value_counts().idxmax())
patient_true_labels = df.groupby("PatientID")["True"].agg(lambda x: x.value_counts().idxmax())

# Combine predictions and true labels
patient_results = pd.DataFrame({
    "Predicted": patient_predictions,
    "True": patient_true_labels
})

# Compute accuracy
accuracy = (patient_results["Predicted"] == patient_results["True"]).mean()

# Print results
print("Patient-level classification results:")
print(patient_results)
print(f"\nPatient-level Accuracy: {accuracy:.2f}")

# Confusion matrix and classification report
conf_matrix = confusion_matrix(patient_results["True"], patient_results["Predicted"])
print("\nConfusion Matrix:")
print(conf_matrix)

report = classification_report(patient_results["True"], patient_results["Predicted"])
print("\nClassification Report:")
print(report)


In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns

# --- Settings ---

transforms_external = transforms.Compose([
    transforms.Resize((245, 457)),
    transforms.ToTensor()
])

# Datasets and DataLoaders
external_data_set = CustomImageDataset(data_set_path="E://Project//PTE//External", transforms=transforms_external)

ex_loader = DataLoader(external_data_set, batch_size=hyper_param_batch, shuffle=True)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_classes = external_data_set.num_classes


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_classes = 3
class_names = ["normal", "NSCLC", "SCC"]

# --- Load Model ---
model = CustomConvNet(num_classes=num_classes)
model.load_state_dict(torch.load('best_best_model.pth', map_location=device))
model.to(device)
model.eval()

# --- Gather Predictions and Labels ---
all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for batch in ex_loader:
        images = batch['image'].to(device)
        labels = batch['label'].cpu().numpy()
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        preds = np.argmax(probs, axis=1)
        all_labels.extend(labels)
        all_preds.extend(preds)
        all_probs.extend(probs)

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
all_probs = np.array(all_probs)

# --- Confusion Matrix ---
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Pretty confusion matrix
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# --- Per-Class Accuracy ---
class_acc = cm.diagonal() / cm.sum(axis=1)
print("\nPer-Class Accuracy:")
for i, acc in enumerate(class_acc):
    print(f"{class_names[i]}: {acc:.2%}")

# --- Classification Report ---
report = classification_report(all_labels, all_preds, target_names=class_names)
print("\nClassification Report:")
print(report)

# --- ROC Curves and AUC ---
labels_binarized = label_binarize(all_labels, classes=np.arange(num_classes))

plt.figure(figsize=(8, 7))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(labels_binarized[:, i], all_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curves')
plt.legend(loc="lower right")
plt.show()


# Gradcam

In [ ]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAMPlusPlus

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# --- Ensure CUDA is available ---
assert torch.cuda.is_available(), "CUDA is not available. Please check your GPU setup."
device = torch.device('cuda')

# --- Load model and move to GPU ---
model = CustomConvNet(num_classes=3)
model.load_state_dict(torch.load('best_best_model.pth'))  # Assuming this was saved from a GPU model
model.to(device).eval()

# --- Set target layer (last convolutional layer) ---
target_layer = model.layer6[0]  # Update if needed

# --- Load and preprocess your RGB image (245x457) ---
# image_path = "testSet//normal//00030 (9).tif"
image_path = "testSet//SCC//00110 (17).tif"
original_image = cv2.imread(image_path)
if original_image is None:
    raise FileNotFoundError(f"Image not found: {image_path}")

original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
rgb_image = np.float32(original_image) / 255.0  # Normalize to [0, 1]

# Convert to tensor and send to GPU
input_tensor = torch.from_numpy(rgb_image.transpose(2, 0, 1)).unsqueeze(0).float().to(device)

# --- Get predicted class ---
with torch.no_grad():
    outputs = model(input_tensor)
    predicted_class = torch.argmax(outputs).item()

# --- Initialize GradCAM on GPU ---
# cam = GradCAM(model=model, target_layers=[target_layer])
cam = GradCAMPlusPlus(model=model, target_layers=[target_layer])

# --- Compute Grad-CAM ---
targets = [ClassifierOutputTarget(predicted_class)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0]

# --- Resize CAM to match original image size ---
grayscale_cam_resized = cv2.resize(grayscale_cam, (original_image.shape[1], original_image.shape[0]))

# --- Overlay CAM on original image ---
visualization = show_cam_on_image(
    rgb_image,
    grayscale_cam_resized,
    use_rgb=True,
    image_weight=0.68,  # more original image
    colormap=cv2.COLORMAP_JET  # default is JET, just ensuring it’s set
)
# --- Show result ---
plt.figure(figsize=(10, 5))
plt.imshow(visualization)
plt.title(f"Grad-CAM (GPU) - Predicted Class: {predicted_class}")
plt.axis('off')
plt.tight_layout()
plt.show()
